# 다이어그램으로 작업하기(Working with Diagrams)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

## 개요(Overview)

역학 시스템 모델링 [Modeling Dynamical Systems](./dynamical_systems.ipynb) 튜토리얼은 Drake 시스템 프레임워크에 대해서 아주 기본적인 소개를 제공했는데 여기에는 여러 시스템을 `Diagram`으로 조립하는 방법도 포함되었다. 이 notebook에서는 다이어그램으로 작업하는 고급/완벽한 개요를 제공한다.

블록 다이어그램(block diagram)은 시스템 이론 및 제어를 표준 모델링 추상화하는 것이다. 이를 통해 매우 복잡한 동적 시스템을 서술하는 모듈식 코드를 작성하기 위한 강력한 추상화를 제공한다.

In [ ]:
import numpy as np
import pydot
from IPython.display import SVG, display
from pydrake.examples import PendulumPlant
from pydrake.systems.controllers import PidController
from pydrake.systems.framework import Diagram, DiagramBuilder, LeafSystem
from pydrake.systems.primitives import AffineSystem, LogVectorOutput

## 다이어그램 만들기 및 시각화하기(Building and visualizing your Diagram)

[introductory tutorial](./dynamical_systems.ipynb)의 `Diagram` 예시를 다시 살펴보며 시작해보자.

In [ ]:
def MakePidControlledPendulum():
    # Use a DiagramBuilder to add and connect the subsystems.
    builder = DiagramBuilder()

    # First add the pendulum.
    pendulum = builder.AddNamedSystem("pendulum", PendulumPlant())

    # Add a PID controller.
    controller = builder.AddNamedSystem("controller",
                                        PidController(kp=[10.], ki=[1.], kd=[1.]))

    # Now "wire up" the controller to the plant.
    builder.Connect(pendulum.get_state_output_port(),
                    controller.get_input_port_estimated_state())
    builder.Connect(controller.get_output_port_control(), pendulum.get_input_port())

    # Make the desired_state input of the controller an input to the diagram.
    builder.ExportInput(controller.get_input_port_desired_state())
    # Make the pendulum state an output from the diagram.
    builder.ExportOutput(pendulum.get_state_output_port())

    # Log the state of the pendulum.
    logger = LogVectorOutput(pendulum.get_state_output_port(), builder)
    logger.set_name("logger")

    pid_controlled_pendulum = builder.Build()
    pid_controlled_pendulum.set_name("PID-controlled Pendulum")

    return pid_controlled_pendulum, pendulum

pid_controlled_pendulum, pendulum = MakePidControlledPendulum()

# Visualize the diagram.
display(
    SVG(
        pydot.graph_from_dot_data(
            pid_controlled_pendulum.GetGraphvizString(
                max_depth=2))[0].create_svg()))


### 시스템 하나를 하나의 DiagramBuilder로 추가하기(You can only add a System to one DiagramBuilder.)

다이어그램 빌더에 하나의 시스템을 추가할 때 중요한 점은 builder가 해당 시스템의 소유권을 가지게 된다는 것이다. 즉, 동일한 시스템을 다른 builder에 추가할 수 없다. 현재 Drake는 C++ 객체 소유권에 대해서 다소 일반적인 오류 메시지를 리포팅한다.:

In [ ]:
second_builder = DiagramBuilder()

try:
    second_builder.AddSystem(pendulum)
except RuntimeError as err:
    print(err)

개념적으로는 간단하지만, Jupyter notebook에서 실수로 발생할 수 있는 문제이다. 한 cell에서 시스템을 정의하고 이 시스템을 다른 셀에서 builder에 추가한 다음, 두 번째 셀을 연속해서 두 번 실행하면 오류가 발생한다. 이러한 이유로, 권장되는 workflow는 시스템을 만들고 즉시 builder에 추가하거나 (같은 셀에서) 위에서와 같이 함수를 사용해서 system을 만드는 것이다.

## Nested diagrams

다이어그램 또한 시스템이며, 서브시스템으로 활용하여 모듈성과 캡슐화를 구현할 수 있다. 다른 context에서 동일한 PID제어 pendulum을 사용하고자 하는데 좌표계를 $\pi$만큼 이동하고 싶다고 가정해 보자. 제어 pendulum 주변에 더 많은 시스템들을 추가함으로써 이를 달성할 수 있다.

In [ ]:
builder = DiagramBuilder()

# We make the PID-controlled pendulum again here, to avoid C++ ownership issues.
pid_controlled_pendulum, pendulum = MakePidControlledPendulum()
builder.AddSystem(pid_controlled_pendulum)

# Shift desired theta by PI with the system y= u + [pi;0]
shift_desired = builder.AddNamedSystem(
    "shift_desired", AffineSystem(D=np.eye(2), y0=[np.pi, 0]))
builder.ExportInput(shift_desired.get_input_port(),
                    "controller_desired_state")

# Connect the shift to the *exported input port* from the subdiagram.
builder.Connect(shift_desired.get_output_port(),
                pid_controlled_pendulum.get_input_port())

# Shift actual theta by -PI.
shift_actual = builder.AddNamedSystem(
    "shift_actual", AffineSystem(D=np.eye(2), y0=[-np.pi, 0]))
builder.Connect(pid_controlled_pendulum.get_output_port(),
                shift_actual.get_input_port())
builder.ExportOutput(shift_actual.get_output_port(), "pendulum_state")

diagram = builder.Build()
diagram.set_name("PID-controlled Pendulum (with θ shifted by π)")

# Visualize the diagram (max depth=1)
display(
    SVG(
        pydot.graph_from_dot_data(
            diagram.GetGraphvizString(max_depth=1))[0].create_svg()))


In [ ]:
# Visualize the diagram (max depth=2)
display(
    SVG(
        pydot.graph_from_dot_data(
            diagram.GetGraphvizString(max_depth=2))[0].create_svg()))


## 서브시스템과 contexts(Subsystems and contexts)

이제 `Diagram`이 있으니, 직접 해당 `Context`를 조작할 수 있다. 하지만 예를 들어 다이어그램의 `Context`에 있는 상태 변수들의 순서는 상황에 따라 취약할 수 있다 (더 많은 시스템들을 `DiagramBuilder`에 추가하면 변경될 수 있음). 게다가 개별 서브시스템은 여러분이 각각의 `Context`를 다루는 데 도움이 되는 메소드를 제공할 수 있다.

`DiagramBuilder::AddSystem()`/`AddNamedSystem()` 메소드가 반환하는 포인터는 여전히 유효한 서브시스템에 대한 포인터이며, 직접 사용할 수 있다. 추가로, `Diagram`은 내부에 포함된 서브시스템에 대한 포인터를 검색하는 메커니즘을 제공한다.

In [ ]:
assert(pid_controlled_pendulum.GetSubsystemByName("pendulum") == pendulum)

더 흥미로운 부분은 서브시스템의 `Context`이다. 다이어그램의 `Context`는 실제로 서브시스템들의 `Context`를 정리해서 모아놓은 것임을 이해해야 한다. 이 서브시스템 `Context`에 직접 접근할 수 있으며, 대부분 [`GetMyContextFromRoot()`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_system.html#ae7fa91d2b2102457ced3361207724e52) 메소드를 사용한다. 다이어그램은 다른 다이어그램의 서브시스템이 될 수 있으므로, 이 메소드는 잠재적으로 중첩된 다이어그램(nested diagram)을 재귀적(recurse)으로 호출되어 올바른 하위 `Context`를 가져온다.

이 예시에서 `Context`들은 다이어그램처럼 중첩(nested)되어 있다:
```
  PID-controlled Pendulum (with θ shifted by π) Context (of a Diagram)
    ↳ PID-controlled Pendulum Context (of a Diagram)
        ↳ pendulum Context
        ↳ controller Context
```

In [ ]:
diagram_context = diagram.CreateDefaultContext()
print(diagram_context)

중요한 점은 이렇게 다이어그램 `Context`에서 검색한 서브시스템 `Context`는 실제로 다이어그램 `Context` 내부의 포인터라는 것이다. 서브시스템 `Context`의 값을 변경하면 루트 `Context`의 값도 함께 변경된다. 사실 이것이 루트 `Context`의 값을 변경하는 권장되는 방법이다.

In [ ]:
pendulum_context = pendulum.GetMyContextFromRoot(diagram_context)
pendulum_context.SetContinuousState([1.2, 0.5])
# The PendulumPlant class provides some helper methods for working with its
# Context.
pendulum_state = pendulum.get_state(pendulum_context)
print(f"θ = {pendulum_state.theta()}, θ̇ = {pendulum_state.thetadot()}")


In [ ]:
# Observe that the root system's Context was also updated.
print(diagram_context)

요약하자면, `LeafSystem`의 `Context`는 시간, 상태, 파라미터, 시스템에 대한 입력을 포함하는 클래스이며, 거의 모든 시스템 메소드에서 접근할 수 있다. `Diagram`의 `Context`는 동일한 추상화를 제공한다. (왜냐하면 `Diagram`도 `System`이기 때문이다). 하지만 추가적으로 서브시스템의 하위 `Context`를 직접 조작하는 방법을 제공한다.


## 입력과 출력 포트를 외부 노출(Exporting input and output ports)

[leaf system 만들기](./authoring_leaf_systems.ipynb)에서 입력 및 출력 포트를 선언한다. `Diagram`을 조립할 때 서브시스템의 입력 및 출력 포트를 노출하고 다이어그램의 입출력을 정의한다. 절대 서브시스템의 포트에 직접 접근하지 말자.

일반적인 실수 중 하나는 서브시스템의 입출력 포트에 직접 연결하려는 시도이다. 다시 중첩된 `Diagram` 예시를 살펴보자:

In [ ]:
builder = DiagramBuilder()

pid_controlled_pendulum, pendulum = MakePidControlledPendulum()
builder.AddSystem(pid_controlled_pendulum)

# Shift desired theta by PI with the system y= u + [pi;0]
shift_desired = builder.AddNamedSystem(
    "shift_desired", AffineSystem(D=np.eye(2), y0=[np.pi, 0]))

# ** WRONG ** This doesn't work, but gives a helpful message.
controller = pid_controlled_pendulum.GetSubsystemByName("controller")
try:
    builder.Connect(shift_desired.get_output_port(),
                    controller.get_input_port_desired_state())
except RuntimeError as err:
    print(err)

위의 예시에서 보았듯이 올바른 접근 방식은 `pid_controlled_pendulum` 서브-다이어그램으로부터 *노출된 입력 포트*로 shift를 연결하는 것이다.

## Diagrams도 시스템이다.(Diagrams are Systems, too)

`Diagram`은 `System` 클래스 인터페이스를 구현한다. 이를 위해 다이어그램은 서브시스템 구현으로 메소드 호출을 분산하고 결과를 수집한다. 다음은 시간 도함수(time derivatives)와 이벤트 게시 작업(publish events)을 수행하는 간단한 예시이다. 하지만 모든 `System` 메소드는 유사하게 수행된다.

In [ ]:
class MyLeafSystem(LeafSystem):
    def __init__(self):
        super().__init__()

        self.DeclareContinuousState(1)
        self.DeclareForcedPublishEvent(self.Publish)
        self.DeclarePeriodicPublishEvent(period_sec=1,
                                         offset_sec=0,
                                         publish=self.Publish)

    def DoCalcTimeDerivatives(self, context, derivatives):
        x = context.get_continuous_state_vector().GetAtIndex(0)
        print(f"{self.get_name()}: DoCalcTimeDerivatives()")
        derivatives.get_mutable_vector().SetAtIndex(0, -x)

    def Publish(self, context):
        print(f"{self.get_name()}: Publish()")

builder = DiagramBuilder()
builder.AddNamedSystem("system1", MyLeafSystem())
builder.AddNamedSystem("system2", MyLeafSystem())
diagram = builder.Build()

context = diagram.CreateDefaultContext()

# To evaluate the time derivatives of the diagram, diagram evaluates the time
# derivatives of the subsystems.
print("diagram.EvalTimeDerivatives()")
diagram.EvalTimeDerivatives(context)

# A ForcedPublish on the diagram calls ForcedPublish on the subsystems.
print("diagram.ForcedPublish()")
diagram.ForcedPublish(context)

## Diagrams and scalar types (double, AutoDiffXd, symbolic::Expression)

Diagrams은 scalar 타입을 지원하며, 모든 서브시스템이 스칼라 타입을 지원한다면 [scalar-type  변환(conversion)](https://drake.mit.edu/doxygen_cxx/group__system__scalar__conversion.html): https://drake.mit.edu/doxygen_cxx/group__system__scalar__conversion.html도 지원한다. 일반적으로 `Diagram`을 기본 scalar로 먼저 빌드한 다음 `ToAutoDiffXd()` 및/또는 `ToSymbolic()`로 변환하는 것이 가장 일반적이다.

In [ ]:
builder = DiagramBuilder()
# AffineSystem is a primitive which supports all scalar types, and
# scalar-conversion.
builder.AddSystem(AffineSystem(y0=[2, 3]))
builder.AddSystem(AffineSystem(D=np.eye(2)))
diagram = builder.Build()

diagram_autodiff = diagram.ToAutoDiffXd()
diagram_symbolic = diagram.ToSymbolic()

In [ ]:
builder = DiagramBuilder()
# MyLeafSystem (defined above) did not implement scalar support, so the
# resulting Diagram will not, either.
builder.AddSystem(MyLeafSystem())
diagram = builder.Build()

print("calling ToAutoDiffXd()")
try:
    diagram_autodiff = diagram.ToAutoDiffXd()
except RuntimeError as err:
    print(err)

print("\ncalling ToSymbolic()")
try:
    diagram_symbolic = diagram.ToSymbolic()
except RuntimeError as err:
    print(err)


이런 지원을 추가하는 방법에 대한 상세한 내용은 [authoring leaf systems](./authoring_leaf_systems.ipynb) 튜토리얼을 참조하자.

# (고급) Diagram으로부터 subsclassing((Advanced) Subclassing from Diagram)

대부분의 사용 사례에서 diagram을 생성하기 위해 `DiagramBuilder()`를 사용하면 충분하다. 하지만 경우에 따라 diagram 자체에 대한 추가 메소드 또는 멤버 변수를 제공하려면 `Diagram`을 상속하여 사용자 정의 클래스를 구현하는 것이 유용할 수 있다.

In [ ]:
class CustomDiagram(Diagram):

    def __init__(self):
        Diagram.__init__(self)
        builder = DiagramBuilder()
        builder = DiagramBuilder()
        self.system1 = builder.AddNamedSystem("system1", MyLeafSystem())
        self.system2 = builder.AddNamedSystem("system2", MyLeafSystem())
        # Instead of builder.Build(), we call
        builder.BuildInto(self)

    def get_system1(self):
        return self.system1
    
    def get_system2(self):
        return self.system2
    
diagram = CustomDiagram()
context = diagram.CreateDefaultContext()
print(context)


Drake에서 한 가지 예시는 `RobotDiagram`이다. 이 클래스는 `MultibodyPlant`와 `SceneGraph`를 자동으로 채워주고 이들을 편리하게 가져올 수 있는 메소드를 제공하는 `Diagram`의 확장 클래스이다.